# Chapter 15. Multiple Regression

In [26]:
from __future__ import division
from collections import Counter
from functools import partial
from linear_algebra import dot, vector_add
from statistics import median, standard_deviation
from probability import normal_cdf
from gradient_descent import minimize_stochastic
from simple_linear_regression import total_sum_of_squares
import math, random

The VP is impressed by your simple regression model, but you know you can do better.  
You start by collecting more data: for each user you get data on how many hours he works each day and whether he has a PhD.  
You can use this additional data to improve your model.  
Accordingly, you hypothesize a linear model with more independent variables:  

$\normalsize \text{minutes} = \alpha + \beta_1 \text{friends} + \beta_2 \text{work hours} + \beta_3 \text{PhD} + \epsilon$  

For the PhD category we can use a dummy variable (see Chapter 11) that equals 1 for users *with* a PhD and 0 for users *without* a PhD.

## The Model

Recall that in Chapter 14 we fit a model of the form:  

$\Large y_i = \alpha + \beta x_i + \epsilon_i$  

Now imagine that each input $\normalsize x_i$ is not a single number, but is instead a vector of $\normalsize k$ numbers $\normalsize \;{x_i}_1, {x_i}_2, \ldots, {x_i}_k$.  
The multiple regression model assumes that:  

$\Large y_i = \alpha + \beta_1{x_i}_1 + \ldots + \beta_k{x_i}_k + \epsilon_i$

In multiple regression the vector of parameters is usually called $\normalsize \beta$.  
We'll want this to include the constant term as well, which we can achieve by adding a column of ones to our data:

and:

Then our model is:

In [27]:
def predict(x_i, beta):
    """ assumes that the first element of each x_i is 1 """
    return dot(x_i, beta)

In this particular case, our independent variable `x` will be a list of vectors, each of which looks like this:

Now for the data that we'll be using:

In [28]:
x = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

## Further Assumptions of the Least Squares Model

There are two further assumptions that are required for this model, as well as our solution, to work.

### Assumption the First

The columns of $x$ are [linearly independent](https://en.wikipedia.org/wiki/Linear_independence), meaning that there is no way to write any one as a weighted sum of some of the others.  
If this assumption fails, there is no reliable way to estimate `beta`.  
To illustrate this in an extreme case, imagine that we have an extra field `num_acquaintances` in our data that, for every user, was exactly equal to `num_friends`.  
Then, starting with `beta`, if we add *any* amount to the `num_friends` coefficient and subtract the same amount from the `num_acquaintances` coefficient, the model's predictions will remain unchanged.  
This means that there is no way to find *the* coefficient for `num_friends`.  
Usually violations of this assumption won't be so obvious.

### Assumption the Second

The columns of $x$ are all uncorrelated with the errors of $\normalsize \epsilon$.  
If this fails to be the case, our estimates of `beta` will be systematically wrong.  
For example, in Chapter 14, we built a model that predicted that each additional friend was associated with an extra 0.90 daily minutes on the site.  
Imagine that it's also the case that:  
- people who work more hours spend less time on the site.
- people with more friends tend to work more hours.
In math terms, imagine that the "actual" model is:  

$\large \text{minutes} = \alpha + \beta_1 \text{friends} + \beta_2 \text{work hours} + \epsilon$  

and that work hours and friends are positively correlated.  
In that case, when we minimize the errors of the single variable model:  

$\large \text{minutes} = \alpha + \beta_1 \text{friends} + \epsilon$.  

we will underestimate $\beta_1$.

Think about what would happen if we made predictions using the single variable model with the "actual" value of $\beta_1$ (the value that arises from minimizing the errors of what we called the "actual" model).  
The predictions would tend to be too small for users who work many hours and too large for users who work few hours, because $\beta_2 > 0$ and we failed to include it.  
Because work hours is positively correlated with number of friends, this means that the predictions tend to be too small for users with many friends and too large for users with few friends.  
The result of this is that we can reduce the errors (in the single-variable model) by decreasing our estimate of $\beta_1$, which means that the error-minimizing $\beta_1$ is smaller than the "actual" value.  
That is, in this case the single-variable least-squares solution is biased to underestimate $\beta_1$.  
And, in general, whenever the independent variables are correlated with the errors like this, our least squares solution will give us a biased estimate of $\beta$.

## Fitting the Model

As we did in the simple linear model, we'll choose `beta` to minimize the sum of squared errors.  
Finding an exact solution is not simple to do by hand, which means we'll need to use gradient descent.  
We'll start by creating an error function to minimize.  
For stochastic gradient descent, we'll just want the squared error corresponding to a single prediction:

In [29]:
def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

def squared_error(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

If you know calculus, you can calculate:

In [30]:
def squared_error_gradient(x_i, y_i, beta):
    """ the gradient (with respect to beta) corresponding to the ith squared error term """
    return [-2 * x_ij * error(x_i, y_i, beta) for x_ij in x_i]

At this point, we're ready to find the optimal beta using stochastic gradient descent:

In [31]:
def estimate_beta(x, y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error,
                               squared_error_gradient,
                               x, 
                               y, 
                               beta_initial,
                               0.001)
random.seed(0)
beta = estimate_beta(x, daily_minutes_good)
beta

[30.625234786488353,
 0.9715448288696535,
 -1.8679272872032218,
 0.911456949921445]

Those results means that our model looks like:  

minutes = 30.63 + 0.972friends - 1.868work hours + 0.911PhD

## Interpreting the Model